In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [7]:
a = ((1, 2), 3)
print(a[0][0])

1


In [8]:
#Angle between points a and c, with midpoint being b
def getAngle(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

#vertical distance between two points (over 0 if ideal is true, under 0 if ideal is false)
def verticalDistance(idealLower, idealUpper):
    return (idealLower[1] - idealUpper[1])

#horizontal distance between two points (over 0 if ideal is true, under 0 if ideal is false)
def horizontalDistance(idealLeft, idealRight):
    return (idealLeft[0] - idealRight[0])

#distance between two points
def distance(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

#is the person doing a y shape?
#@params:
    #leftAngle: angle between left wrist and left shoulder
    #rightAngle: angle between right wrist and right shoulder
    #rightEyeToRightWrist: distance between right eye and right wrist
    #leftEyeToLeftWrist: distance between left eye and left wrist
    #wristDistance: distance between left wrist and right wrist
    #leftWristToElbow: distance between left wrist and left elbow
    #rightWristToElbow: distance between right wrist and right elbow
#@return: True if the person is doing a y shape, False otherwise
# def isY(leftAngle, rightAngle, rightEyeToRightWrist, leftEyeToLeftWrist, wristDistance, leftWristToElbow, rightWristToElbow):
def isY(rightWrist, leftWrist, rightElbow, leftElbow, rightShoulder, leftShoulder, rightEye, leftEye):
    
    wristToElbow = max(distance(leftWrist, leftElbow) * 0.8, distance(rightWrist, rightElbow) * 0.8)
    leftAngle = getAngle(leftShoulder, leftElbow, leftWrist)
    rightAngle = getAngle(rightShoulder, rightElbow, rightWrist)


    if (leftAngle > 100 and leftAngle < 180 and rightAngle > 180 and rightAngle < 250):
        if (distance(rightEye, rightWrist) > 0 and distance(leftEye, leftWrist) > 0):
            if (distance(leftWrist, rightWrist) > wristToElbow):
                if (not isM(leftWrist, rightWrist, rightElbow, leftElbow, rightShoulder, leftShoulder, rightEye, leftEye) and not isC(rightWrist, leftWrist, rightEye, leftEye)):
                    return True
    return False

#is the person doing an m shape?
#@params:
    #leftAngle: angle between left wrist and left shoulder
    #rightAngle: angle between right wrist and right shoulder
    #rightWristToShoulder: distance between right wrist and right shoulder
    #leftWristToShoulder: distance between left wrist and left shoulder
    #wristDistance: distance between left wrist and right wrist
    #leftWristToElbow: distance between left wrist and left elbow
    #rightWristToElbow: distance between right wrist and right elbow
#@return: True if the person is doing an m shape, False otherwise

def isM(leftWrist, rightWrist, rightElbow, leftElbow, rightShoulder, leftShoulder, rightEye, leftEye):
    # newWristToElbow = max(leftWristToElbow, rightWristToElbow)
    if (verticalDistance(rightShoulder, rightWrist) > 0 and verticalDistance(leftShoulder, leftWrist) > 0):
        if (rightAngle > 240 and leftAngle < 120):
            if (distance(leftWrist, leftEye) < (distance(leftEye, leftElbow) * 1.3) and distance(rightWrist, rightEye) < (distance(rightEye, rightElbow) * 1.3)):
                return True
    return False

def isC(rightWrist, leftWrist, rightEye, leftEye):
    vertBetweenWrists = abs(verticalDistance(rightWrist, leftWrist))
    if (vertBetweenWrists > 0.1):
        if((horizontalDistance(rightWrist, rightEye) > 0 and horizontalDistance(leftWrist, leftEye) > 0) or (horizontalDistance(rightWrist, rightEye) < 0 and horizontalDistance(leftWrist, leftEye) < 0)):
            return True
    return False
        
def isA(rightWrist, leftWrist, rightShoulder, leftShoulder, rightElbow, leftElbow):
    if (verticalDistance(rightShoulder, rightWrist) > 0 ) and (verticalDistance(leftShoulder, leftWrist) > 0):
        if (rightWrist[0] > rightShoulder[0] and leftWrist[0] < leftShoulder[0]):
            rightAngle = getAngle(rightShoulder, rightElbow, rightWrist)
            leftAngle = getAngle(leftShoulder, leftElbow, leftWrist)
            if (190 < rightAngle < 240 and 120 < leftAngle < 165):
                return True
    return False



    


In [9]:

clicked = False
def click(event, x, y, flags, params):
  print("hi")
  if (event == cv2.EVENT_LBUTTONDOWN):
    global clicked
    if not clicked:
      clicked = True
    else:
      clicked = False

cap = cv2.VideoCapture(2)
curLetter = ""
position = (0, 0, 0, 0)
x = y = 0
isxyAssigned = False
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    print(x, y)
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print("right shoulder:", results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y)
    try:
      right_shoulder = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y])
      left_shoulder = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
      right_elbow = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW].y])
      left_elbow = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y])
      right_wrist = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y])
      left_wrist = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y])
      left_eye = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE].y])
      right_eye = np.array([results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].x, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE].y])
      rightAngle = getAngle(right_shoulder, right_elbow, right_wrist)
      leftAngle = getAngle(left_shoulder, left_elbow, left_wrist)
      # print("rightAngle:", rightAngle, "leftAngle:", leftAngle, "wristDistance:", distance(right_wrist, left_wrist))
      
      if (isY(right_wrist, left_wrist, right_elbow, left_elbow, right_shoulder, left_shoulder, right_eye, left_eye)):
        curLetter = "Y"
      elif (isM(left_wrist, right_wrist, right_elbow, left_elbow, right_shoulder, left_shoulder, right_eye, left_eye)):
        curLetter = "M"
      elif (isC(right_wrist, left_wrist, right_eye, left_eye)):
        curLetter = "C"
      elif (isA(right_wrist, left_wrist, right_shoulder, left_shoulder, right_elbow, left_elbow)):
        curLetter = "A"
      else:
        curLetter = ""
      
    except:
      print("No pose detected")
      curLetter = ""

    textsize = cv2.getTextSize(curLetter, cv2.FONT_HERSHEY_SIMPLEX, 4, 2)[0]
    
    if clicked:
      mp_drawing.draw_landmarks(
          image,
          results.pose_landmarks,
          mp_pose.POSE_CONNECTIONS,
          landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    image = cv2.flip(image, 1)

    image = cv2.putText(image, curLetter, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 4, (0, 0, 255), 2, cv2.LINE_AA)
    # print(type(textsize))
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('YMCA Detector', image)

    cv2.setMouseCallback('YMCA Detector', click, [clicked])

    x = (image.shape[1] - textsize[0])//2
    y = (image.shape[0] + textsize[1])//2
    if (not isxyAssigned):
      position = cv2.getWindowImageRect("YMCA Detector")
    
      isxyAssigned = True

    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

2022-01-15 19:22:51.455 python[61026:1076942] simpleDALPluginMain
2022-01-15 19:22:51.457 python[61026:1076942] SimpleDALPlugin: QueryInterface(plugin:uuid:interface:): 
2022-01-15 19:22:51.457 python[61026:1076942] SimpleDALPlugin: Release(plugin:): 
2022-01-15 19:22:51.457 python[61026:1076942] SimpleDALPlugin: InitializeWithObjectID(plugin:objectID:): 
2022-01-15 19:22:51.601 python[61026:1076942] SimpleDALPlugin: ObjectGetPropertyData(plugin:objectID:address:qualifiedDataSize:qualifiedData:dataSize:dataUsed:data:): Optional(1969841184)
2022-01-15 19:22:51.601 python[61026:1076942] SimpleDALPlugin: ObjectGetPropertyData(plugin:objectID:address:qualifiedDataSize:qualifiedData:dataSize:dataUsed:data:): Optional(1953653102)
2022-01-15 19:22:51.601 python[61026:1076942] SimpleDALPlugin: ObjectSetPropertyData(plugin:objectID:address:qualifiedDataSize:qualifiedData:dataSize:data:): 
2022-01-15 19:22:51.601 python[61026:1076942] SimpleDALPlugin: ObjectSetPropertyData(plugin:objectID:addres

0 0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


No pose detected
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
hi
639 402
639 402
639 402
639 402
hi
639 402
639 402
hi
639 402
639 402
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
639 402
hi
639 402
639 402
hi
639 402
639 402
597 402
639 402
639 402
639 402
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
hi
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
597 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
639 402
597 402
597 402
639 402
603 402
603 402
603 402
639 402
603 402
639 402
639 402
603 402
639 402
603 402
639 40

KeyboardInterrupt: 